In [ ]:
# trying videos

In [ ]:
import cv2
import moviepy.editor as mp
import urllib.request
from gtts import gTTS
from IPython.display import Audio
import nltk
from nltk.tokenize import sent_tokenize
from io import BytesIO
from pydub import AudioSegment

AudioSegment.ffmpeg = "C:\manal\Python8\Lib\site-packages\ffmpeg"
nltk.download('punkt')
    
def createGenVideoSlide (prompt, prs, output_pptx, ImagesCounter, slideLayout) :
    user_input = get_prompt(prompt)
    if user_input == "done":
        return user_input, ImagesCounter
    # create a loop until done
    speak.Speak("Ok, I am working on it, bear with me, please")
    response_txt = getGenText (output_pptx+": " + user_input)      

    sentences = sent_tokenize(response_txt)
    print(sentences)
    
    image_paths = response_revised_prompts = []
    for i in range (len(sentences)):
        image_path, response_revised_prompt = getGenImage(output_pptx+": Please create a sequence of images for a video assembly for the following prompt: " + sentences[i], ImagesCounter)
        image_paths.append(image_path)
        response_revised_prompts.append(response_revised_prompt)
        ImagesCounter=ImagesCounter+1

    # Generate the entire audio
    tts = gTTS(response_txt, lang='en')
    audioFileName = output_pptx+"audio_"+str(ImagesCounter)+".mp3"
    tts.save(audioFileName)
    audio_fp = BytesIO()
    tts.write_to_fp(audio_fp)
    audio_fp.seek(0)
    audio = AudioSegment.from_mp3(audioFileName)
    #audio = AudioSegment.from_file(audio_fp, format="mp3")

    # Get the duration of the entire audio in milliseconds
    audio_length = len(audio)

    # Split audio into segments for each sentence
    sentence_audio_lengths = [audio_length * len(sentence) / len(script) for sentence in sentences]

    audio_segments = []
    start_time = 0
    for length in sentence_audio_lengths:
        end_time = start_time + length
        segment = audio[start_time:end_time]
        audio_segments.append(segment)
        start_time = end_time

    # Load images and audio segments
    images = [cv2.imread(image_paths[i]) for i in range(len(image_paths))]
    
    #Audio(audioFileName)
    videoFileName = output_pptx+"video_"+str(ImagesCounter)+".avi"

    
    # Create video frames
    height, width, layers = images[0].shape
    video = cv2.VideoWriter(videoFileName, cv2.VideoWriter_fourcc(*'DIVX'), 30, (width, height))

    # Create a silent moviepy video clip from images
    image_clips = []
    for i, image in enumerate(images):
        frame_duration = len(audio_segments[i]) / 1000  # duration in seconds
        clip = mp.ImageClip(image_paths[i]).set_duration(frame_duration)
        image_clips.append(clip)

    # Concatenate image clips to form the final video clip
    video_clip = mp.concatenate_videoclips(image_clips, method="compose")

    # Add audio segments to the video clip
    final_audio = mp.concatenate_audioclips([mp.AudioFileClip(BytesIO(segment.export(format="mp3").read()), fps=segment.frame_rate) for segment in audio_segments])
    final_clip = video_clip.set_audio(final_audio)

    # Write final video to file
    final_clip.write_videofile(videoFileName, codec="libx264")



    speak.Speak("Ok, done")    
    createMediaSlide (prs, output_pptx, response_txt, videoFileName, response_revised_prompt, slideLayout)
    slideLayout=slideLayout+1
    if slideLayout > 4:
        slideLayout = 1

    return user_input, ImagesCounter, slideLayout, response_txt, image_path, response_revised_prompt

In [ ]:
tts = gTTS("Hello, this is me tired and want to finish in a few minutes pelase", lang='en')
audioFileName = "Test1.mp3"
tts.save(audioFileName)

In [ ]:
pydub.utils.get_prober_name = lambda: "/absolute/path/to/ffprobe"

In [ ]:
audio_fp = BytesIO()
tts.write_to_fp(audio_fp)
audio_fp.seek(0)
audio = AudioSegment.from_mp3(audioFileName)